In [4]:
from teste_cepespdata import teste_cepespdata

In [8]:
inst = teste_cepespdata(2018,5, agregacao_regional=6)

Ano: 2018
Cargo: 5 
Banco Teste: False



In [9]:
inst.call_teste_geral()

Starting test...


In [11]:
inst.qtde_votos_total_1turno

206055134

In [12]:
inst.qtde_comparecimento_1turno

234223144